##### Whale optimization algorithm

##### import libraries

In [59]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import math


##### initialise the whale population

In [71]:
def initialise_whale(num_whales,dim):
    whales= set()
    while len(whales) < num_whales:
        temp = tuple(random.uniform(-5, 5) for _ in range(dim))
        whales.add(temp)
    
    whales = list(whales)
    normalised_whales = [list(whale) for whale in whales]
    
    for i in range(dim):
        x = [whale[i] for whale in whales]
        minx = min(x)
        maxx = max(x)
        for n in range(num_whales):
            normalised_whales[n][i] = (whales[n][i] - minx) / (maxx - minx) if maxx != minx else 0.0
    
    return [whales], normalised_whales

##### Initialise the fitness function

In [72]:
def fitness_function(x):
    fit_val=0
    for i in x:
        fit_val+=i**2
    return fit_val

##### Find Xbest

In [73]:
def find_best(a,x):
    f_best=a[a.index(min(a))]
    x_best=x[a.index(min(a))]
    return f_best,x_best

##### visual

In [81]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Function to plot whale movement and return the frame
def plot_whale_movement_frame(whale_pos, iter_num):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_zlim(-5, 5)
    ax.grid(True)
    # # Plot the initial positions
    # initial_positions = np.array(whale_pos[0])
    # ax.scatter(initial_positions[:, 0], initial_positions[:, 1], initial_positions[:, 2], color='r', label='Initial Positions')

    # Plot the positions at the current iteration
    iter_positions = np.array(whale_pos[iter_num])
    ax.scatter(iter_positions[:, 0], iter_positions[:, 1], iter_positions[:, 2], color='b', label='Iteration {}'.format(iter_num))
    
    # # Plot the paths of all whales up to the current iteration
    # for whale in range(len(whale_pos[0])):
    #     path = np.array([iter_pos[whale] for iter_pos in whale_pos[:iter_num+1]])
    #     ax.plot(path[:, 0], path[:, 1], path[:, 2], linestyle='--')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Whale Optimization Algorithm Movement')
    ax.legend()

    # Convert plot to an image array
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    return frame

# Function to create a video from a list of frames
def create_video_from_frames(frames, video_filename, fps=10):
    height, width, layers = frames[0].shape
    video = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for frame in frames:
        video.write(frame)

    video.release()

# Example usage


In [82]:
maxx=5
minx=-5
no_of_whales=30
dim=3
b = 1 #spiral coefficient
whale_pos, nor_whale_pos=initialise_whale(no_of_whales,dim)
fitness_values=[]
for i in whale_pos[0]:
    fitness_values.append(fitness_function(i))
f_best, x_best=find_best(fitness_values,whale_pos[0])
max_iter=100
iter=0
X_best=[]
fit_v_iter=[]
fit_v_iter.append(fitness_values)
fit_best=[]
X_best.append(x_best)
fit_best.append(f_best)
while(iter<max_iter):
    a = 2 * (1 - iter / max_iter)
    x_new_iter=[]
    for whale in range(no_of_whales):
        
        A = 2 * a * random.random() - a
        C = 2 * random.random()
        l = random.uniform(-1, 1)
        p = random.random()
        
        if(p<0.5):
            if(A<1):
                dnew=[abs(C*x_best[i]-whale_pos[iter][whale][i]) for i in range(dim)]
                xnew=[x_best[i]-A*dnew[i] for i in range(dim)]
                x_new_iter.append(xnew)
            else:
                index=random.randint(0,no_of_whales-1)
                dnew=[abs(C*whale_pos[iter][index][i]-whale_pos[iter][whale][i]) for i in range(dim)]
                xnew=[whale_pos[iter][index][i]-A*dnew[i] for i in range(dim)]
                x_new_iter.append(xnew)
        else:
            dnew=[abs(C*x_best[i]-whale_pos[iter][whale][i]) for i in range(dim)]
            xnew=[dnew[i] * math.exp(b * l) * math.cos(2 * math.pi * l) + x_best[i] for i in range(dim)]
            x_new_iter.append(xnew)
    for i in range(no_of_whales):
        for j in range(dim):
            if(x_new_iter[i][j]<minx):
                x_new_iter[i][j]=minx
            elif(x_new_iter[i][j]>maxx):
                x_new_iter[i][j]=maxx
    whale_pos.append(x_new_iter)
    iter+=1
    fitness_values=[]
    for i in whale_pos[iter]:
        fitness_values.append(fitness_function(i))
    fit_v_iter.append(fitness_values)
    f_best_iter, x_best_iter=find_best(fitness_values,whale_pos[iter])
    if(f_best_iter<f_best):
        f_best=f_best_iter
        x_best=x_best_iter
        X_best.append(x_best)
        fit_best.append(f_best)
# Example usage
video_filename = 'whale_optimization_algorithm.mp4'

# Create frames for each iteration
num_iterations = len(whale_pos)  # Assuming whale_pos contains positions across all iterations
frames = []
for iter_num in range(num_iterations):
    frame = plot_whale_movement_frame(whale_pos, iter_num)
    frames.append(frame)

# Create video from frames
create_video_from_frames(frames, video_filename)


In [76]:
x_best

[8.248133726739493e-18, 1.8718403792415846e-17, -6.42854406898069e-18]

In [77]:
fit_best

[1.370127666096601,
 1.0831338745728512,
 0.6568530724369862,
 0.19395235848151152,
 0.07446094074647637,
 0.02278711583816203,
 0.016682772405863737,
 0.011860958461938145,
 0.0062983534349911755,
 0.0017904046721171837,
 0.0012791963069071991,
 0.0006036149336993159,
 0.00048486300424356143,
 0.00028380231572542594,
 0.00024137920872606425,
 9.367550001944855e-05,
 7.191547630609024e-05,
 3.674861147174617e-05,
 3.2417215132163174e-05,
 1.5025730443717314e-05,
 1.2488566654244611e-05,
 7.922942341197705e-06,
 4.088682703039229e-06,
 1.6535624129423237e-06,
 3.2310514579257334e-07,
 1.136567582700346e-07,
 7.125281574741458e-08,
 2.2015701338566743e-08,
 7.74183737964264e-09,
 3.1635866047739943e-09,
 1.4904760069080845e-09,
 5.450031742863539e-10,
 1.250865743486096e-10,
 9.896893136203426e-11,
 2.6484673197185975e-11,
 1.215129231510348e-11,
 3.554793002863468e-12,
 1.2189967357448793e-12,
 1.130012031632176e-12,
 3.418656080914947e-13,
 1.445581147683368e-13,
 7.297783430720784e-14

In [78]:
X_best

[(0.08375210874440775, -0.12082856539008269, 1.1612552295525909),
 [-0.36718968807772107, -0.5325437144604244, 0.8152930759738679],
 [-0.515328836987204, -0.6167728998025725, -0.10430940645755693],
 [-0.430984681428844, 0.07435825952265696, 0.0517243858924451],
 [-0.2530362402039081, 0.08049001630297699, 0.0628884660768014],
 [-0.08819922762757199, 0.10216070145733942, 0.06761067343109473],
 [-0.10198888861198066, 0.07449242849507812, 0.02705396648298382],
 [-0.013889128127707248, 0.09642859644837909, 0.04867829463719871],
 [-0.05297312890294688, 0.058591051307716725, -0.0076999841480002995],
 [-0.034740931408425, 0.023442831446299814, 0.005822886807475301],
 [-0.035201693098744956, -0.004003939901664779, -0.004899548464177866],
 [-0.01650998907606631, 0.007911842500128241, 0.01638407588668496],
 [-0.01653023479991473, -0.0007134006886134574, 0.014529466650938542],
 [-0.013217425485282973, 0.006461547177862081, 0.008206728174777127],
 [-0.014400556680082126, 0.0054875416089831125, 0.00

In [83]:
whale_pos[99]

[[-2.0800171668596316e-16, -7.091320266104013e-17, -1.1587978048078068e-16],
 [-3.5942357362606554e-17, 9.35958791913364e-17, -1.754128136402345e-16],
 [-1.6585002984738092e-17, 1.148610783486839e-16, -4.7840497401029154e-17],
 [-2.459210294894431e-17, 9.954712984138473e-17, -9.052986369247461e-17],
 [-7.099399222074769e-17, 1.7505752294286967e-17, -6.287375153413506e-16],
 [-2.5964694927564628e-17, 1.0226017960483193e-16, -9.691225522762753e-17],
 [-2.5344462381166603e-17, 1.0268455178830678e-16, -1.0391474577748038e-16],
 [3.127577169833784e-17, 1.4881296085550452e-16, -8.102861372505265e-17],
 [3.145343306514065e-17, 1.0172764081419301e-16, 3.0229293153353277e-16],
 [-4.29693823585965e-17, 9.077136157967055e-17, -2.1463015608365327e-16],
 [7.996462835130383e-17, 1.5082730614364241e-16, -8.29312009788642e-17],
 [-2.715567841808066e-17, 9.907445110726265e-17, -1.0820772015771472e-16],
 [-2.9276989110392834e-17, 9.865399059778507e-17, -1.232597837904423e-16],
 [-2.786623651743721e-17, 